In [ ]:
import nltk
nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn
nltk.download("subjectivity")
from nltk.corpus import movie_reviews
from sklearn.metrics import classification_report


mr = movie_reviews


from nltk.corpus import subjectivity
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer, VaderConstants

analyzer = SentimentIntensityAnalyzer()
doc = subjectivity.sents()

In [ ]:
def lol2str(doc):
    # flatten & join
    return " ".join([w for sent in doc for w in sent])

def score_sent(document, use_pos=False):
    pos = []
    neg = []
    obj = []
    for sent in document:
        if use_pos:
            tagged_sent = pos_tag(sent, tagset='universal')
        else:
            tagged_sent = [(w, None) for w in sent]

        for tok, tag in tagged_sent:
            ss = lesk(sent, tok, pos=pos2wn.get(tag, None))
            if ss:
                sense = swn.senti_synset(ss.name())
                pos.append(sense.pos_score())
                neg.append(sense.neg_score())
                obj.append(sense.obj_score())
    return pos, neg, obj

def subjectivity_sentence_level(document, analyzer):
    S = 0
    O = 0
    labels = ['S', 'O']
    for sentence in document:
        value = analyzer.polarity_scores(" ".join(sentence))
        if value["compound"] != 0:
            S+=1
        else:
            O+=1
    return labels[np.argmax(np.asarray([S, O]))]
def rm_objective_sentences(document, analyzer):
    new_doc = []
    ob_doc = []
    for sentence in document:
        value = analyzer.polarity_scores(" ".join(sentence))
        if value["compound"] != 0:
            new_doc.append(" ".join(sentence))
        else:
            ob_doc.append(" ".join(sentence))
    return new_doc, ob_doc
def polarity_doc_level(document, analyzer):
    value = analyzer.polarity_scores(document)
    P = 0
    N = 0
    labels = ['P', 'N']
    if value["compound"] > 0:
        P += 1
    elif value["compound"] <= 0: # In this way we penalize the neg class
        N += 1
    return labels[np.argmax(np.asarray([P, N]))]

In [ ]:
# Train and test with Stratified K Fold

skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
scores_clf = []
scores_subjectivity = []


#corpus = [lol2str(d) for d in rev_neg] + [lol2str(d) for d in rev_pos]
#vectors = vectorizer.fit_transform(mr)

ref_sub, ref_ob = rm_objective_sentences(doc, analyzer)

ref = numpy.array([0] * len(ref_ob) + [1] * len(ref_sub))

for i, (train_index, test_index) in enumerate(skf.split(doc, ref)):
    x_train, x_test = [doc[indx] for indx in train_index], [doc[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]

    vectorizer = TfidfVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    scores = cross_validate(clf, vectors, labels, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    hyp_sentence = [subjectivity_sentence_level(doc, analyzer) for doc in test_x_split]
    scores_subjectivity.append(f1_score(y_test, hyp_sentence, average='macro'))
        
    
print('F1 classifier:', round(sum(scores_clf)/len(scores_clf), 3))
print('F1 Subjectivity:',  round(sum(scores_subjectivity)/len(scores_subjectivity), 3))

In [ ]:
scores_clf = []
scores_subjectivity = []

ref_sub, ref_ob = rm_objective_sentences(mr, analyzer)

ref = numpy.array([0] * len(ref_ob) + [1] * len(ref_sub))
new_mr = []

for i, (train_index, test_index) in enumerate(skf.split(mr, ref)):
    x_train, x_test = [mr[indx] for indx in train_index], [mr[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]

    vectorizer = TfidfVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    scores = cross_validate(clf, vectors, labels, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    new_mr.append(hyp)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    hyp_sentence = [subjectivity_sentence_level(mr, analyzer) for mr in test_x_split]
    scores_subjectivity.append(f1_score(y_test, hyp_sentence, average='macro'))
        
    
print('F1 classifier:', round(sum(scores_clf)/len(scores_clf), 3))
print('F1 Subjectivity:',  round(sum(scores_subjectivity)/len(scores_subjectivity), 3))

In [ ]:
# Train and test with Stratified K Fold

scores_clf = []
scores_polarity = []


rev_wo_ob_neg = new_mr.paras(categories='neg')
rev_wo_ob_pos = new_mr.paras(categories='pos')
#corpus = [lol2str(d) for d in rev_neg] + [lol2str(d) for d in rev_pos]
#vectors = vectorizer.fit_transform(mr)

ref = numpy.array([0] * len(rev_wo_ob_neg) + [1] * len(rev_wo_ob_pos))

for i, (train_index, test_index) in enumerate(skf.split(new_mr, ref)):
    x_train, x_test = [new_mr[indx] for indx in train_index], [new_mr[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]

    vectorizer = TfidfVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    scores = cross_validate(clf, vectors, ref, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    hyp_polarity = [polarity_doc_level(new_mr, analyzer) for new_mr in x_test]
    scores_polarity.append(f1_score(y_test, hyp_polarity, average='macro'))

    
    
print('F1 classifier Polarity w/o Obj:', round(sum(scores_clf)/len(scores_clf), 3))
print('F1 Polarity:',  round(sum(scores_polarity)/len(scores_polarity), 3))



In [ ]:
scores_clf = []
scores_polarity = []

rev_neg = mr.paras(categories='neg')
rev_pos = mr.paras(categories='pos')
#corpus = [lol2str(d) for d in rev_neg] + [lol2str(d) for d in rev_pos]
#vectors = vectorizer.fit_transform(mr)

ref = numpy.array([0] * len(rev_neg) + [1] * len(rev_pos))

for i, (train_index, test_index) in enumerate(skf.split(mr, ref)):
    x_train, x_test = [mr[indx] for indx in train_index], [mr[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]

    vectorizer = TfidfVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    scores = cross_validate(clf, vectors, ref, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    hyp_polarity = [polarity_doc_level(mr, analyzer) for mr in x_test]
    scores_polarity.append(f1_score(y_test, hyp_polarity, average='macro'))

    
    
print('F1 classifier Polarity with Obj:', round(sum(scores_clf)/len(scores_clf), 3))
print('F1 Polarity:',  round(sum(scores_polarity)/len(scores_polarity), 3))

In [ ]:
def extract(doc):
    aspects = []
    asp_doc = []
    for sent in doc.sents:
        target = None
        opinion = None
        for tok in sent:
            if tok.dep_ == 'nsubj' and tok.pos_ == 'NOUN':
                target = tok.text
            if tok.pos_ == 'ADJ':
                descr = ''
                for child in tok.children:
                    if child.pos_ != 'ADV':
                        continue
                    descr += child.text + ' '
                opinion = descr + tok.text
        if target:
            aspects.append((target, opinion))
            asp_doc.append(opinion)
    return aspects, asp_doc
            
aspects, asp_doc = extract(doc)

In [ ]:
rev_neg = asp_doc.paras(categories='neg')
rev_pos = asp_doc.paras(categories='pos')
#corpus = [lol2str(d) for d in rev_neg] + [lol2str(d) for d in rev_pos]
#vectors = vectorizer.fit_transform(mr)

ref = numpy.array([0] * len(rev_neg) + [1] * len(rev_pos))

for i, (train_index, test_index) in enumerate(skf.split(mr, ref)):
    x_train, x_test = [asp_doc[indx] for indx in train_index], [asp_doc[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]

    vectorizer = TfidfVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    scores = cross_validate(clf, vectors, ref, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    hyp_polarity = [polarity_doc_level(asp_doc, analyzer) for asp_doc in x_test]
    scores_polarity.append(f1_score(y_test, hyp_polarity, average='macro'))

    
    
print('F1 classifier Polarity with Obj:', round(sum(scores_clf)/len(scores_clf), 3))
print('F1 Polarity:',  round(sum(scores_polarity)/len(scores_polarity), 3))